In [143]:
import download
from data_utils import load_CIFAR10
import data_utils


In [152]:
url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
CIFAR10_data = "./data"
download.maybe_download_and_extract(url,CIFAR10_data)

Data has apparently already been downloaded and unpacked.


  # Load the raw CIFAR-10 data
  

In [145]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
 cifar10_dir = './data/cifar-10-batches-py'
 X_train, y_train, X_test, y_test = data_utils.load_CIFAR10(cifar10_dir)
 try:
       del X_train, y_train
       del X_test, y_test
       print('Clear previously loaded data.')
 except:
       pass

 X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

 # subsample the data
 mask_val = list(range(num_training, num_training + num_validation))
 X_val = X_train[mask_val]
 y_val = y_train[mask_val]


 mask_train = list(range(num_training))
 X_train = X_train[mask_train]
 y_train = y_train[mask_train]

 mask_test = list(range(num_test))
 X_test = X_test[mask_test]
 y_test = y_test[mask_test]

 mask_dev = np.random.choice(num_training, num_dev, replace=False)
 X_dev = X_train[mask_dev]
 y_dev = y_train[mask_dev]
  
 # Preprocessing: reshape the image data into rows
 X_train = np.reshape(X_train, (X_train.shape[0], -1))
 X_val = np.reshape(X_val, (X_val.shape[0], -1))
 X_test = np.reshape(X_test, (X_test.shape[0], -1))
 X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
 
# Normalize the data: subtract the mean image
 mean_image = np.mean(X_train, axis = 0)
 X_train -= mean_image
 X_val -= mean_image
 X_test -= mean_image
 X_dev -= mean_image   

# add bias dimension and transform into columns
 X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
 X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
 X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
 X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
 return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev

# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Clear previously loaded data.
Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


# SoftmaxClassifier

In [146]:
import numpy as np
from random import shuffle
from past.builtins import xrange

def softmax_loss_naive(W, X, y, reg):
  """
  Softmax loss function, naive implementation (with loops)

  Inputs have dimension D, there are C classes, and we operate on minibatches
  of N examples.

  Inputs:
  - W: A numpy array of shape (D, C) containing weights.
  - X: A numpy array of shape (N, D) containing a minibatch of data.
  - y: A numpy array of shape (N,) containing training labels; y[i] = c means
    that X[i] has label c, where 0 <= c < C.
  - reg: (float) regularization strength

  Returns a tuple of:
  - loss as single float
  - gradient with respect to weights W; an array of same shape as W
  """
  # Initialize the loss and gradient to zero.
  loss = 0.0
  dW = np.zeros_like(W)

  #############################################################################
  # TODO: Compute the softmax loss and its gradient using explicit loops.     #
  # Store the loss in loss and the gradient in dW. If you are not careful     #
  # here, it is easy to run into numeric instability. Don't forget the        #
  # regularization!                                                           #
  #############################################################################
  num_train = X.shape[0]
  for i in xrange(num_train):
      sco = np.dot(X[i], W)
      exp_sco = np.exp(sco)
      sum_sco = np.sum(exp_sco)
      loss -= np.log(exp_sco[y[i]] / sum_sco)
      dW += np.dot(X[i].T[:, np.newaxis], exp_sco[np.newaxis, :]) / sum_sco
      dW[:, y[i]] -= X[i].T
  loss = loss/X.shape[0] + reg*np.sum(W**2)
  dW = dW/X.shape[0] + 2*reg*W
  #############################################################################
  #                          END OF YOUR CODE                                 #
  #############################################################################

  return loss, dW


In [147]:
import time

In [148]:
# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.428974
sanity check: 2.302585


In [149]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: -3.247471 analytic: -3.247471, relative error: 5.672236e-09
numerical: 0.612269 analytic: 0.612269, relative error: 5.191434e-08
numerical: -1.099376 analytic: -1.099376, relative error: 1.624255e-08
numerical: 2.080537 analytic: 2.080537, relative error: 2.353169e-08
numerical: -2.296465 analytic: -2.296465, relative error: 1.606693e-08
numerical: 0.420825 analytic: 0.420825, relative error: 2.079602e-07
numerical: 2.327426 analytic: 2.327426, relative error: 5.762871e-09
numerical: -0.931625 analytic: -0.931625, relative error: 2.727510e-08
numerical: -1.492394 analytic: -1.492394, relative error: 1.812192e-08
numerical: 1.851247 analytic: 1.851247, relative error: 3.532736e-08
numerical: -0.601768 analytic: -0.601768, relative error: 1.901265e-08
numerical: -0.944304 analytic: -0.944304, relative error: 3.199467e-09
numerical: 0.812426 analytic: 0.812426, relative error: 8.655273e-08
numerical: -1.989485 analytic: -1.989485, relative error: 1.445387e-09
numerical: 1.88597

In [150]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.


def softmax_loss_vectorized(W, X, y, reg):
  """
  Softmax loss function, vectorized version.

  Inputs and outputs are the same as softmax_loss_naive.
  """
  # Initialize the loss and gradient to zero.
  loss = 0.0
  dW = np.zeros_like(W)

  #############################################################################
  # TODO: Compute the softmax loss and its gradient using no explicit loops.  #
  # Store the loss in loss and the gradient in dW. If you are not careful     #
  # here, it is easy to run into numeric instability. Don't forget the        #
  # regularization!                                                           #
  #############################################################################
  '''
  sco = np.dot(X, W)
  sco[np.arange(sco.shape[0])] -= np.reshape(np.max(sco, axis=1),[-1, 1])
  exp_sco = np.exp(sco)
  sum_sco = np.sum(exp_sco) 
  softmax_sco = exp_sco / sum_sco
  cross_entropy = np.sum(-np.log(softmax_sco))
  loss = cross_entropy/X.shape[0] + reg*np.sum(W**2)

  dsco = exp_sco / (exp_sco.sum(axis=1)[:,np.newaxis])
  dsco[range(X.shape[0]), y] -= 1
  dsco /= X.shape[0]
  dW = np.dot(X.T, dsco) + 2*reg*W
  '''

  sco = np.dot(X, W)
  correct_sco = sco[range(X.shape[0]), y]
  exp_ = np.exp(sco - correct_sco[:, np.newaxis])
  log_ = np.log(exp_.sum(axis=1))
  loss = np.sum(log_) / X.shape[0]
  loss += reg * np.sum(W**2)

  dsco = exp_ / (exp_.sum(axis=1)[:, np.newaxis])
  dsco[range(X.shape[0]), y] -= 1
  dsco /= X.shape[0]
  dW = np.dot(X.T, dsco) + 2*reg*W
  #############################################################################
  #                          END OF YOUR CODE                                 #
  #############################################################################

  return loss, dW

In [151]:
# The two versions should compute the same results, but the vectorized version should be
# much faster.

tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))


tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

naive loss: 2.428974e+00 computed in 0.024018s
vectorized loss: 2.428974e+00 computed in 0.002976s
Loss difference: 0.000000
Gradient difference: 0.000000
